In [1]:
import pandas as pd
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
import pandas as pd

In [2]:
data = pd.read_csv("/kaggle/input/stocknews/Combined_News_DJIA.csv")

In [3]:
train = data[data['Date'] < '2015-01-01']
test = data[data['Date'] > '2014-12-31']

In [4]:
len(test),len(train)

(378, 1611)

In [5]:
trainheadlines = []
for row in range(0,len(train.index)):
    trainheadlines.append(' '.join(str(x) for x in train.iloc[row,2:27]))

In [6]:
trainvect = CountVectorizer()

In [7]:
Trainfeature = trainvect.fit_transform(trainheadlines)

In [8]:
####Detailed view of Document Count Matrix
DTM_With_Colm = pd.DataFrame(Trainfeature.toarray(),columns= trainvect.get_feature_names())

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [9]:
Trainfeature.shape

(1611, 31675)

In [10]:
Logis = LogisticRegression()

In [11]:
Model1 = Logis.fit(Trainfeature,train['Label'])

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [12]:
Model1

LogisticRegression()

In [13]:
testheadlines =[]
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))

In [14]:
len(testheadlines)

378

In [15]:
Testfeature = trainvect.transform(testheadlines)

In [16]:
Testfeature.shape

(378, 31675)

In [17]:
Predicted = Model1.predict(Testfeature)

In [18]:
pd.crosstab(test["Label"], Predicted , rownames=["Actual"] , colnames= ["Predict"])

Predict,0,1
Actual,,
0,61,125
1,92,100


In [19]:
from sklearn.naive_bayes import MultinomialNB

Nb = MultinomialNB()

In [20]:
Model2 = Nb.fit(Trainfeature,train['Label'])

In [21]:
Nbpredicted = Model2.predict(Testfeature)

In [22]:
Nbpredicted.shape

(378,)

In [23]:
pd.crosstab(test["Label"], Nbpredicted, rownames= ["Acutal"],colnames=["Predict"])

Predict,0,1
Acutal,,
0,46,140
1,51,141


In [24]:
import numpy as np
from sklearn.metrics import accuracy_score
y_NaviBayes = Nbpredicted
y_true = test["Label"]
accuracy_score(y_NaviBayes,y_true)
x_Logist = Predicted
x_true = test["Label"]
accuracy_score(x_Logist,x_true)

0.42592592592592593

In [25]:
advvect = CountVectorizer(ngram_range=(1,2))
%time
advancedtrain = advvect.fit_transform(trainheadlines)
advancedtrain.shape

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.39 µs


(1611, 398396)

In [26]:
advmodel = MultinomialNB()
advancemodel = advmodel.fit(advancedtrain,train["Label"])
advancetest = advvect.transform(testheadlines)
advNBprediction = advmodel.predict(advancetest) 
advNBprediction.shape

(378,)

In [27]:
pd.crosstab(test["Label"],advNBprediction, rownames=["Acutal"],colnames=["Predicted"])

Predicted,0,1
Acutal,,
0,3,183
1,0,192


In [28]:
x_adNB = advNBprediction
x_test = test["Label"]

In [29]:
accuracy_score(x_test,x_adNB)

0.5158730158730159

In [30]:
from sklearn import metrics
metrics.accuracy_score(x_test,x_adNB)

0.5158730158730159

In [31]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim

In [32]:
%time
tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
    
#Our Document
trainheadlines

# list for tokenized documents in loop
texts = []

# loop through document list
for i in trainheadlines:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
    # add tokens to list
    texts.append(stemmed_tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
    
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.39 µs


In [33]:
%time
#generate LDA
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word = dictionary, passes=1,chunksize=10000,update_every=1)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.58 µs


In [34]:
ldamodel.print_topics(5)

[(6,
  '0.026*"b" + 0.016*"s" + 0.004*"kill" + 0.004*"will" + 0.004*"year" + 0.004*"protest" + 0.004*"us" + 0.004*"new" + 0.004*"polic" + 0.004*"say"'),
 (5,
  '0.019*"s" + 0.017*"b" + 0.006*"say" + 0.004*"us" + 0.004*"year" + 0.004*"will" + 0.004*"kill" + 0.004*"new" + 0.003*"polic" + 0.003*"russia"'),
 (2,
  '0.010*"s" + 0.004*"will" + 0.004*"us" + 0.004*"say" + 0.004*"year" + 0.004*"kill" + 0.004*"world" + 0.004*"peopl" + 0.003*"b" + 0.003*"protest"'),
 (4,
  '0.045*"b" + 0.019*"s" + 0.005*"say" + 0.005*"year" + 0.005*"israel" + 0.005*"us" + 0.004*"new" + 0.004*"world" + 0.004*"govern" + 0.004*"kill"'),
 (8,
  '0.019*"s" + 0.006*"us" + 0.005*"say" + 0.005*"kill" + 0.004*"govern" + 0.004*"new" + 0.004*"polic" + 0.003*"year" + 0.003*"russia" + 0.003*"world"')]

In [35]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
news = pyLDAvis.gensim.prepare(ldamodel,corpus, dictionary)

/opt/conda/lib/python3.7/site-packages/pyLDAvis/_prepare.py:248: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [36]:
news

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.017875  0.000794       1        1  26.138635
9      0.002774 -0.004449       2        1  23.957316
1      0.009229  0.000243       3        1  16.966285
5      0.001264 -0.002665       4        1  11.353919
0     -0.008716 -0.004625       5        1   7.297670
8     -0.010816 -0.007014       6        1   5.057471
3     -0.000046  0.000367       7        1   4.903959
6      0.004846  0.003281       8        1   3.578623
7      0.000815  0.006073       9        1   0.528083
2     -0.017225  0.007995      10        1   0.218040, topic_info=       Term          Freq         Total Category  logprob  loglift
18        b  12962.000000  12962.000000  Default  30.0000  30.0000
136       s   8619.000000   8619.000000  Default  29.0000  29.0000
171      us   2440.000000   2440.000000  Default  28.0000  28.0000
139     say   2620.000000   2620.000000  Default  27.0000  27.0000
89     kill   2060.000000   2060.000000  Default  26.0000  26.0000
..      ...           ...           ...      ...      ...      ...
261   polic      2.456668   1568.317154  Topic10  -6.1145  -0.3307
167       u      2.438627   1540.513589  Topic10  -6.1219  -0.3202
18        b      3.779098  12962.965326  Topic10  -5.6838  -2.0121
17   attack      2.296519   1387.547823  Topic10  -6.1819  -0.2757
83   israel      2.281184   2106.592815  Topic10  -6.1886  -0.6999

[860 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
0         1  0.299062   000
0         2  0.235096   000
0         3  0.134578   000
0         4  0.146208   000
0         5  0.037383   000
...     ...       ...   ...
1689      5  0.079884  zone
1689      6  0.031954  zone
1689      7  0.039942  zone
1689      8  0.063907  zone
1689      9  0.007988  zone

[2273 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 10, 2, 6, 1, 9, 4, 7, 8, 3])